In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
!pip install statsforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 914.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.2 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.1.8-py3-none-any.whl size=158200 sha256=985ea2cfc9501f1c0967a79f4f19db932c4c1698b0599408615813251cfd5665
  Stored in directory: /root/.cache/pip/wheels/a4/2b/3e/8ac985ad100a8f27de940864344fe14f47bc3d2fed7f29bf70
Successfully built fugue-sql-antlr


In [ ]:
!pip install datasetsforecast

In [ ]:
!pip install mlforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
import statsforecast
from statsforecast import StatsForecast
from statsforecast.models import Naive, SeasonalNaive, WindowAverage, SeasonalWindowAverage

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:25: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/time series hw/coconut oil.csv')

In [ ]:
df.head()

,Date,Coconut_oil
0,1960M01,"390,00"
1,1960M02,"379,00"
2,1960M03,"361,00"
3,1960M04,"338,00"
4,1960M05,"321,00"


In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%YM%m')
df['Coconut_oil ($/mt)'] = df['Coconut_oil '].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
fig = px.line(df, x='Date', y="Coconut_oil ($/mt)", title='Coconut Oil price changes')
fig.show()

In [ ]:
df.rename(columns={"Date": "ds", "Coconut_oil ($/mt)": "y"}, inplace=True)

In [ ]:
df.drop(columns=['Coconut_oil '], inplace=True)

In [ ]:
df['unique_id'] = ['coconut oil' for x in range(df.shape[0])]

## Cross Validation

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

## Naive Model

In [ ]:
from statsforecast.models import AutoETS, AutoARIMA, ARIMA, GARCH

boosting = xgb.XGBRegressor()

models = [ARIMA((1, 0, 1)), GARCH(1, 1)]

sf = StatsForecast(
    df = df,
    models = models,
    freq = 'M',
    n_jobs = -1
)

In [ ]:
crossvalidation_df = sf.cross_validation(
    df = df,
    h = 1, #forecast horizon
    test_size = 20,
    n_windows = None
  )

In [ ]:
crossvalidation_df.head()

,ds,cutoff,y,ARIMA,"GARCH(1,1)"
unique_id,,,,,
coconut oil,2022-02-28,2022-01-31,2230.219971,2140.710938,967.625977
coconut oil,2022-03-31,2022-02-28,2094.610107,2231.520020,984.558594
coconut oil,2022-04-30,2022-03-31,1813.329956,2042.048584,998.173523
coconut oil,2022-05-31,2022-04-30,1700.500000,1736.704346,1005.995972
coconut oil,2022-06-30,2022-05-31,1540.530029,1672.573730,1011.748352


In [ ]:
crossvalidation_df.rename(columns = {'y' : 'actual'}, inplace = True) # rename actual values

cutoff = crossvalidation_df['cutoff'].unique()


for k in range(len(cutoff)):
    cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
    StatsForecast.plot(df, cv.loc[:, cv.columns != 'cutoff'], engine="plotly")

In [ ]:
crossvalidation_df

,ds,cutoff,actual,ARIMA,"GARCH(1,1)"
unique_id,,,,,
coconut oil,2022-02-28,2022-01-31,2230.219971,2140.710938,967.625977
coconut oil,2022-03-31,2022-02-28,2094.610107,2231.520020,984.558594
coconut oil,2022-04-30,2022-03-31,1813.329956,2042.048584,998.173523
coconut oil,2022-05-31,2022-04-30,1700.500000,1736.704346,1005.995972
coconut oil,2022-06-30,2022-05-31,1540.530029,1672.573730,1011.748352
coconut oil,2022-07-31,2022-06-30,1384.569946,1489.993652,1014.926941
coconut oil,2022-08-31,2022-07-31,1248.099976,1342.739502,1015.976807
coconut oil,2022-09-30,2022-08-31,1108.099976,1211.062256,1015.435547
coconut oil,2022-10-31,2022-09-30,1173.250000,1071.282227,1013.552063


In [ ]:
fig = px.line(crossvalidation_df, x='ds', y=crossvalidation_df.columns[2:], title='Coconut Oil price changes')
fig.update_layout(legend=dict(title_font_family="Times New Roman",
                              font=dict(size= 23)
))
fig.show()

In [ ]:
from datasetsforecast.losses import (mae, mape, mase, rmse, smape)

for model in ['ARIMA', 'GARCH(1,1)']:

    maex = mae(crossvalidation_df['actual'], crossvalidation_df[model])
    print(f'MAE using cross-validation for {model}:  {maex}')
    rmsex = rmse(crossvalidation_df['actual'], crossvalidation_df[model])
    print(f'RMSE using cross-validation for {model}:  {rmsex}')
    mapex = mape(crossvalidation_df['actual'], crossvalidation_df[model])
    print(f'MAPE using cross-validation for {model}:  {mapex}')
    print('\n')

MAE using cross-validation for ARIMA:  69.40132141113281
RMSE using cross-validation for ARIMA:  87.54100799560547
MAPE using cross-validation for ARIMA:  5.018216371536255


MAE using cross-validation for GARCH(1,1):  297.7913513183594
RMSE using cross-validation for GARCH(1,1):  475.8708190917969
MAPE using cross-validation for GARCH(1,1):  18.041154742240906




## ML

In [ ]:
df

,ds,y,unique_id
0,1960-01-01,390.00,coconut oil
1,1960-02-01,379.00,coconut oil
2,1960-03-01,361.00,coconut oil
3,1960-04-01,338.00,coconut oil
4,1960-05-01,321.00,coconut oil
...,...,...,...
761,2023-06-01,1012.73,coconut oil
762,2023-07-01,1047.38,coconut oil
763,2023-08-01,1099.09,coconut oil
764,2023-09-01,1071.67,coconut oil


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
X = df['ds']
y = df['y']

In [ ]:
import xgboost as xgb

In [ ]:
import lightgbm as lgb

from mlforecast import MLForecast
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [ ]:
mlf = MLForecast(
    models = [LinearRegression(), lgb.LGBMRegressor(), XGBRegressor()],
    lags=[1, 12],
    freq = 'M'
)

In [ ]:
df1 = df.copy()
df1

,ds,y,unique_id
0,1960-01-01,390.00,coconut oil
1,1960-02-01,379.00,coconut oil
2,1960-03-01,361.00,coconut oil
3,1960-04-01,338.00,coconut oil
4,1960-05-01,321.00,coconut oil
...,...,...,...
761,2023-06-01,1012.73,coconut oil
762,2023-07-01,1047.38,coconut oil
763,2023-08-01,1099.09,coconut oil
764,2023-09-01,1071.67,coconut oil


In [ ]:
tscv = TimeSeriesSplit(n_splits=20, test_size=1)
predictors = []

for i in range(df1.shape[0] - 20, df1.shape[0]):
    df_cv = df[:i]
    print(df_cv)
    mlf.fit(df_cv)
    predictors.append(mlf.predict(1))

            ds        y    unique_id
0   1960-01-01   390.00  coconut oil
1   1960-02-01   379.00  coconut oil
2   1960-03-01   361.00  coconut oil
3   1960-04-01   338.00  coconut oil
4   1960-05-01   321.00  coconut oil
..         ...      ...          ...
741 2021-10-01  1922.86  coconut oil
742 2021-11-01  1960.68  coconut oil
743 2021-12-01  1695.71  coconut oil
744 2022-01-01  2016.14  coconut oil
745 2022-02-01  2147.89  coconut oil

[746 rows x 3 columns]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 490
[LightGBM] [Info] Number of data points in the train set: 734, number of used features: 2
[LightGBM] [Info] Start training from score 684.216716
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

In [ ]:
predictors

[     unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-02-28       2142.578213    1758.264862    1592.49585,
      unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-03-31       2223.211859    1842.118708   1636.944946,
      unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-04-30       2079.397093    1921.399167   1607.640625,
      unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-05-31       1790.025242    1900.080252   1654.270142,
      unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-06-30       1677.986517    1657.697158   1733.279053,
      unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-07-31       1517.994697    1608.125285   1471.869751,
      unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor
 0  coconut oil 2022-08-31      

In [ ]:
crossvalidation_df2 = crossvalidation_df.copy()

In [ ]:
crossvalidation_df2['LinearRegression'] = [float(predictors[i]['LinearRegression']) for i in range(len(predictors))]
crossvalidation_df2['LGBMRegressor'] = [float(predictors[i]['LGBMRegressor']) for i in range(len(predictors))]
crossvalidation_df2['XGBRegressor'] = [float(predictors[i]['XGBRegressor']) for i in range(len(predictors))]
crossvalidation_df2['hybrid'] = (0.8*crossvalidation_df2['ARIMA'] + 0.2*crossvalidation_df2['XGBRegressor'])

In [ ]:
crossvalidation_df2.head()

,ds,cutoff,actual,ARIMA,"GARCH(1,1)",LinearRegression,LGBMRegressor,XGBRegressor
unique_id,,,,,,,,
coconut oil,2022-02-28,2022-01-31,2230.219971,2140.710938,967.625977,2142.578213,1758.264862,1592.495850
coconut oil,2022-03-31,2022-02-28,2094.610107,2231.520020,984.558594,2223.211859,1842.118708,1636.944946
coconut oil,2022-04-30,2022-03-31,1813.329956,2042.048584,998.173523,2079.397093,1921.399167,1607.640625
coconut oil,2022-05-31,2022-04-30,1700.500000,1736.704346,1005.995972,1790.025242,1900.080252,1654.270142
coconut oil,2022-06-30,2022-05-31,1540.530029,1672.573730,1011.748352,1677.986517,1657.697158,1733.279053


In [ ]:
fig = px.line(crossvalidation_df2, x='ds', y=crossvalidation_df2.columns[2:], title='Coconut Oil price changes')
fig.update_layout(legend=dict(title_font_family="Times New Roman",
                              font=dict(size= 23)
))
fig.show()

In [ ]:
from datasetsforecast.losses import (mae, mape, mase, rmse, smape)

for model in ['ARIMA', 'LinearRegression', 'LGBMRegressor', 'XGBRegressor', 'hybrid']:

    maex = mae(crossvalidation_df2['actual'], crossvalidation_df2[model])
    print(f'MAE using cross-validation for {model}:  {maex}')
    rmsex = rmse(crossvalidation_df2['actual'], crossvalidation_df2[model])
    print(f'RMSE using cross-validation for {model}:  {rmsex}')
    mapex = mape(crossvalidation_df2['actual'], crossvalidation_df2[model])
    print(f'MAPE using cross-validation for {model}:  {mapex}')
    print('\n')

MAE using cross-validation for ARIMA:  69.40132141113281
RMSE using cross-validation for ARIMA:  87.54100799560547
MAPE using cross-validation for ARIMA:  5.018216371536255


MAE using cross-validation for LinearRegression:  79.02202937812929
RMSE using cross-validation for LinearRegression:  100.1748931065928
MAPE using cross-validation for LinearRegression:  5.706350244756096


MAE using cross-validation for LGBMRegressor:  118.09263509089783
RMSE using cross-validation for LGBMRegressor:  159.10283383205552
MAPE using cross-validation for LGBMRegressor:  8.219882924497373


MAE using cross-validation for XGBRegressor:  157.43212890625
RMSE using cross-validation for XGBRegressor:  216.09175656076445
MAPE using cross-validation for XGBRegressor:  11.028732456258956


MAE using cross-validation for hybrid:  66.3440563964844
RMSE using cross-validation for hybrid:  85.77196762907656
MAPE using cross-validation for hybrid:  4.928475109157254


